# **Book Recommendation System**

## **1. Overview**

### 1.1 Data
There are 3 datasets:
- **Books**: contains book information including book title, author, publication year, and publisher.
- **Users**: contains user information including user ID, user location and age if available.
- **Ratings**: contains book rating which is expressed on a scale of 1-10 with higher value indicates higher appreciation.
### 1.2 Objective
- Perform EDA to gain insights into the datasets.
- Create algorithm to reccomend book to users.



## **2. Prepare Data**

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


### 2.1 Books Dataset

In [90]:
# Load data
books = pd.read_csv("Books.csv", usecols=[0,1,2,3,4])

# Inspect data
books.head()

c:\Users\Phuong\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [91]:
# Data info
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


Notice that column year-of-publication is of object type, which should be converted to integer type.

There are some null values but we will leave it as it is.

In [92]:
# Convert year of publication to integer
book = books.copy()
# Cooerce the conversion will return NaN value if value cannot be converted into numerical value
book['Year-Of-Publication'] = pd.to_numeric(book['Year-Of-Publication'], errors='coerce')

# Find all the rows that year of publication is not an integer
books[book['Year-Of-Publication'].isnull()]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


There are 3 data points where the values of book author, year of publication and publisher are mismatched.\
To fix this, we move the value from book author to year of publication and year of publication to publisher. Also, the author name is concatenated at the end of the book-title so we will extract this information and put it in the correct column.

In [93]:
mask = books['Year-Of-Publication'].isin(['DK Publishing Inc','Gallimard'])

# Extract the book-author from book-title
authors = []
titles = []
for title in books.loc[mask,'Book-Title']:  
    author = title.split(';')[-1].split('"')[0]
    book_title = title.split(';')[0].split('\\')[0]

    authors.append(author)
    titles.append(book_title)

# Switch the values
books.loc[mask, ['Year-Of-Publication', 'Publisher']] = books.loc[mask, ['Book-Author', 'Year-Of-Publication']].values
books.loc[mask, 'Book-Author']  = authors
books.loc[mask, 'Book-Title'] = titles

# Convert Year Of Publication column to integer type
books['Year-Of-Publication'] = pd.to_numeric(books['Year-Of-Publication'])

books[books['ISBN'].isin(['078946697X','2070426769','0789466953'])]

### 2.2 Users Dataset

In [27]:
# Load data
users = pd.read_csv("Users.csv")

# Inpsect data
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [28]:
# Users info
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


There are 110762 missing values in age column out of 278858 values. Since we will use content-based filtering based on book information so we can leave the missing value as it is.

### 2.3 Ratings Dataset


In [101]:
# Load data
ratings = pd.read_csv("Ratings.csv")

# Inpsect data
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [102]:
# Ratings info
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [110]:
# Confirm that every user id in ratings dataset also exist in user dataset
len(ratings[~ratings['User-ID'].isin(users['User-ID'])]['User-ID'].unique()) == 0

True